<a href="https://colab.research.google.com/github/shinyoung260/AI-Therapist-Healing-Space/blob/main/Healing_Space.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Application Overview
-  Patients click a button to start typing
-  The application detect patients has finished typing and entering
-  The patients messages passed to the Gemini model, which streams back a response
-  After Gemini finishes typing the patient message bar is reactiviated
-  All previous messages from both the user and Gemini, is displayed in a
chatbot component

## Install the Python SDK

In [109]:
!pip install -U -q "google-generativeai>=0.7.2"

In [110]:
! pip install --upgrade gradio

In [111]:
pip install google-generativeai google-cloud-storage

In [112]:
# Configure the SDK with your API key
import google.generativeai as genai
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [113]:
model = genai.GenerativeModel('models/gemini-1.5-flash')
response = model.generate_content("what is popular Korean food?")
print(response.text)

Korean cuisine is incredibly diverse, but some of the most popular dishes both in Korea and internationally include:

**Main Courses:**

* **Bibimbap:** A mixed rice dish with sautéed and seasoned vegetables, meat (often beef), a fried egg, and gochujang (Korean chili paste).  Many variations exist.
* **Bulgogi:** Thinly sliced marinated beef, usually grilled.  A classic Korean BBQ staple.
* **Kimchi jjigae (Kimchi stew):** A spicy and savory stew made with kimchi, tofu, pork or other meats, and vegetables.
* **Doenjang jjigae (Fermented soybean paste stew):** A hearty stew with doenjang (fermented soybean paste), vegetables, and often tofu or meat.
* **Korean BBQ (Korean style grilled meats):**  This is a broad category encompassing various meats (beef, pork, chicken) marinated and grilled at the table.  Often includes numerous banchan (side dishes).


**Side Dishes (Banchan):**

* **Kimchi:** Fermented Napa cabbage, a staple in Korean cuisine, with countless variations.
* **Kimchi Je

## Select Question text, Answer text, and Topic

In [114]:
import pandas as pd
import io

df = pd.read_csv('20200325_counsel_chat.csv')
selected_column = df[['topic', 'questionText', 'answerText']]

# Save to a new CSV file
selected_column.to_csv("counsel_chat.csv", index=False)
df1 = pd.read_csv("counsel_chat.csv")

# Convert DataFrame to a list of dictionaries (required format)
df1.to_json("counsel_chat.json", orient="records", indent=4)

FileNotFoundError: [Errno 2] No such file or directory: '20200325_counsel_chat.csv'

## Convert JSON file to JSONL format

In [ ]:
import json

# Load the JSON file
with open("counsel_chat.json", "r") as f:
    json_data = json.load(f)

# Write the data to a JSONL file
with open("counsel_chat.jsonl", "w") as jsonl_output:
    for entry in json_data:
        json.dump(entry, jsonl_output)
        jsonl_output.write("\n")

In [ ]:
import json

with open("counsel_chat.jsonl", "r") as f:
    for _ in range(5):  # Print first 5 lines
        print(json.loads(f.readline()))

# Clean duplicate questions

In [ ]:
import json

# Load the JSONL file
with open("counsel_chat.jsonl", "r") as f:
    lines = f.readlines()

# Parse JSONL lines
json_data = [json.loads(line) for line in lines]

# Remove duplicates and normalize text
cleaned_data = []
seen_questions = set()

for entry in json_data:
    question = entry['questionText'].strip()
    answer = entry['answerText'].strip()

    if question not in seen_questions:
        seen_questions.add(question)
        cleaned_data.append({
            'topic': entry['topic'],
            'questionText': question,
            'answerText': answer
        })

# Write cleaned data to a new JSONL file
with open("cleaned_counsel_chat.jsonl", "w") as jsonl_output:
    for entry in cleaned_data:
        json.dump(entry, jsonl_output)
        jsonl_output.write("\n")

print("Dataset cleaned and saved to cleaned_counsel_chat.jsonl")

# UI & UX

In [ ]:
import time
import gradio as gr

chat = model.start_chat(history=[])

# Transform Gradio history to Gemini format
def transform_history(history):
    new_history = []
    for chat in history:
        new_history.append({"parts": [{"text": chat[0]}], "role": "user"})
        new_history.append({"parts": [{"text": chat[1]}], "role": "model"})
    return new_history

def response(message, history):
    global chat
    # The history will be the same as in Gradio, the 'Undo' and 'Clear' buttons will work correctly.
    chat.history = transform_history(history)
    response = chat.send_message(message)
    response.resolve()

    # Each character of the answer is displayed
    for i in range(len(response.text)):
        time.sleep(0.05)
        yield response.text[: i+1]

gr.ChatInterface(response,
                 title='Healing Space',
                 textbox=gr.Textbox(placeholder="Chat with me")).launch(debug=True)